In [1]:
import utils.ip2country

IP2LOCATION-LITE-DB1.CSV upload can take up to a minute!


In [2]:
import os
import pathlib
pcap_fnames = [str(pathlib.Path(root) / file) for root, subdirs, files in os.walk('./pcaps') for file in files if file.endswith('.pcap')]
pcap_fnames.sort()
print(pcap_fnames[:5])

[]


In [3]:
def fname_to_day_and_honeypot_ip(fname):  # TODO move to utils
    # 'pcaps/2019-09-05_to_172.31.21.94.pcap' -> ['2019-09-05', '172.31.21.94']
    return fname.replace('.pcap', '').split('/')[1].split('_to_')

In [32]:
from pypacker import ppcap
from pypacker.layer12 import ethernet
from pypacker.layer3 import ip
from pypacker.layer4 import tcp
from pypacker.layer4 import udp
from pypacker.layer3 import icmp
from collections import Counter

In [25]:
day_country_stats = Counter()
icmp_type_stats = Counter()

for fname in pcap_fnames:
    day, honeypot_ip = fname_to_day_and_honeypot_ip(fname)
    preader = ppcap.Reader(filename=fname)
    for ts, buf in preader:
        eth = ethernet.Ethernet(buf)
        if eth[ethernet.Ethernet, ip.IP] is not None:
            if eth[ip.IP].src_s == honeypot_ip:
                continue  # skip packets emitted by honeypot
            if eth[icmp.ICMP] is not None:
                # see https://gitlab.com/mike01/pypacker/-/blob/master/pypacker/layer3/icmp.py
                if eth[icmp.ICMP].type not in {icmp.ICMP_ECHO_REPLY, icmp.ICMP_UNREACH, icmp.ICMP_ECHO}:
                    icmp_type_stats[eth[icmp.ICMP].type] += 1
                    src_ip = eth[ip.IP].src_s
                    country = utils.ip2country.get_ip_country(src_ip)
                    if not country:
                        continue  # skip packets with unknown country
                    day_country_stats[f'{day} {country}'] += 1

In [31]:
day_country_stats.most_common(30)

[('2020-01-04 IN', 267),
 ('2021-01-25 BR', 197),
 ('2020-03-16 IN', 193),
 ('2020-01-03 IN', 176),
 ('2020-03-17 IN', 140),
 ('2020-01-04 BD', 118),
 ('2020-03-16 ID', 112),
 ('2020-01-04 ID', 99),
 ('2020-03-18 IN', 92),
 ('2020-03-14 US', 76),
 ('2020-01-03 BD', 70),
 ('2020-03-16 BD', 69),
 ('2020-03-17 ID', 66),
 ('2020-01-03 ID', 62),
 ('2020-02-01 US', 60),
 ('2020-01-31 US', 57),
 ('2020-12-17 BR', 53),
 ('2020-03-15 IN', 51),
 ('2020-03-11 US', 46),
 ('2020-03-17 BD', 43),
 ('2020-03-15 ID', 40),
 ('2020-03-18 ID', 40),
 ('2020-01-04 AU', 35),
 ('2020-01-31 ZA', 35),
 ('2020-03-24 US', 34),
 ('2020-03-18 BD', 33),
 ('2020-12-18 BR', 33),
 ('2020-12-17 US', 32),
 ('2020-03-16 US', 31),
 ('2020-02-01 ZA', 30)]

In [30]:
for dc in day_country_stats:
    if 'IR' in dc:
        print(dc, day_country_stats[dc])

2019-09-28 IR 1
2019-11-02 IR 1
2019-11-21 IR 4
2019-11-26 IR 1
2019-12-11 IR 1
2020-01-21 IR 3
2020-02-16 IR 1
2020-03-06 IR 1
2020-03-07 IR 1
2020-03-09 IR 1
2020-03-14 IR 3
2020-03-15 IR 5
2020-09-08 IR 1
2020-10-23 IR 9
2020-10-26 IR 9
2020-10-28 IR 8
2020-12-17 IR 3
2020-12-18 IR 1
2021-01-12 IR 1
2021-01-23 IR 1
2021-01-25 IR 1
2021-02-23 IR 3
2021-04-16 IR 4
2021-07-13 IR 1
2021-07-21 IR 1
2021-08-11 IR 1
